<style>
    @media print{
        body {
            position:relative !important;
        }
        .celltag_new_page {
            page-break-before: always !important;
        }
    }
</style>
# COMPSCI 371 Homework 10

### Problem 0 (3 points)

## Part 1: Mathematics of Correlation and Convolution

### Problem 1.1 (Exam Style)

### Problem 1.2 (Exam Style)

## Part 2: Coding Correlation and Convolution

In [1]:
from urllib.request import urlretrieve
from os import path as osp


def retrieve(file_name, semester='fall22', course='371', homework=10):
    if osp.exists(file_name):
        print('Using previously downloaded file {}'.format(file_name))
    else:
        fmt = 'https://www2.cs.duke.edu/courses/{}/compsci{}/homework/{}/{}'
        url = fmt.format(semester, course, homework, file_name)
        urlretrieve(url, file_name)
        print('Downloaded file {}'.format(file_name))

In [2]:
retrieve('oracle.py')

Using previously downloaded file oracle.py


In [3]:
from oracle import oracle, show


def test(function, x=[1, 2, 3, 4], y=[5, 6]):
    for operator in ('correlate', 'convolve'):
        for mode in ('full', 'same', 'valid'):
            for f, g in ((x, y), (y, x)):
                c = function(operator, f, g, mode)
                show(operator, f, g, mode, c)

### Problem 2.1

In [4]:
def dot(a, b): 
    p_sum = 0
    for i in range(0, min(len(a), len(b))):
        p_sum += a[i] * b[i]
    return p_sum 

def flip(s):
    return s[::-1]

In [5]:
def valid_convolution(a, b):
    #ell is shorter array, s is longer array 
    ell = a
    s = b
    if(len(a) < len(b)): 
        s = a
        ell = b
    
    convolution = []
    for i in range(0, len(ell) - len(s) + 1): 
        convolution.append(dot(ell[i: i+len(s)], flip(s)))
    return convolution

In [6]:
a, b = [1, 2, 3, 4], [5, 6]
print(valid_convolution(a, b))
print(valid_convolution(b, a))

[16, 27, 38]
[16, 27, 38]


### Problem 2.2

We will use valid_convolution() to determing whether short left padding or long left padding

In [7]:
#In short left padding, we add 1 zero to the front, two to the back, and run valid_convolution()
a = [1,2,3,4,5,6,7]
b = [8,9,10,11]

a_padded = [0,1,2,3,4,5,6,7,0,0]
print(valid_convolution(a_padded, b))

[52, 90, 128, 166, 204, 178, 136]


In [8]:
#In long left padding, we add 2 zero to the front, one to the back, and run valid_convolution()
a = [1,2,3,4,5,6,7]
b = [8,9,10,11]

a_padded = [0,0,1,2,3,4,5,6,7,0]
print(valid_convolution(a_padded, b))

[25, 52, 90, 128, 166, 204, 178]


In [9]:
c = oracle('convolve', a, b, 'same')
show('convolve', a, b, 'same', c)

convolve([1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11], same) = [25, 52, 90, 128, 166, 204, 178]


We see that the results given in our $oracle()$ function matches with the long left padding. 

### Problem 2.3

In [10]:
def pad(array, left, right): 
    copy = []
    for i in array: 
        copy.append(i)
    for i in range(0, left): 
        copy.insert(0, 0)
    for i in range(0, right): 
        copy.append(0)
    return copy

In [29]:
import math

def apply(operation, a, b, mode):
    ell = a
    s = b
    if(len(a) < len(b)): 
        s = a
        ell = b

    if operation == 'convolve': 
        if mode == 'full': 
            ell_padded = pad(ell, math.ceil(len(s) / 2), math.floor(len(s) / 2))
            return valid_convolution(ell_padded, s)
        elif mode == 'same': 
            ell_padded = pad(ell, math.ceil((len(s) - 1) / 2), math.floor((len(s) - 1) / 2))
            return valid_convolution(ell_padded, s)
        elif mode == 'valid':
            return valid_convolution(ell, s)  
    elif operation == 'correlate': 
        if mode == 'same': 
            ell_padded = pad(ell, math.ceil((len(s) - 1) / 2), math.floor((len(s) - 1) / 2))
            if(len(b) > len(a)): 
                return apply('convolve', s, flip(ell_padded), 'valid')
            else: 
                return apply('convolve', ell_padded, flip(s), 'valid')
            
        return apply('convolve', a, flip(b), mode)

In [30]:
test(apply)

correlate([1, 2, 3, 4], [5, 6], full) = [6, 17, 28, 39, 20]
correlate([5, 6], [1, 2, 3, 4], full) = [20, 39, 28, 17, 6]
correlate([1, 2, 3, 4], [5, 6], same) = [6, 17, 28, 39]
correlate([5, 6], [1, 2, 3, 4], same) = [39, 28, 17, 6]
correlate([1, 2, 3, 4], [5, 6], valid) = [17, 28, 39]
correlate([5, 6], [1, 2, 3, 4], valid) = [39, 28, 17]
convolve([1, 2, 3, 4], [5, 6], full) = [5, 16, 27, 38, 24]
convolve([5, 6], [1, 2, 3, 4], full) = [5, 16, 27, 38, 24]
convolve([1, 2, 3, 4], [5, 6], same) = [5, 16, 27, 38]
convolve([5, 6], [1, 2, 3, 4], same) = [5, 16, 27, 38]
convolve([1, 2, 3, 4], [5, 6], valid) = [16, 27, 38]
convolve([5, 6], [1, 2, 3, 4], valid) = [16, 27, 38]


## Part 3: Back-Propagation through a Convolution 

### Problem 3.1 (Exam Style)

$$
\begin{eqnarray*}
c_0 &=& a_0 b_0 \\
c_1 &=& a_0 b_1 + a_1 b_0 \\
c_2 &=& a_0 b_2 + a_1 b_1 + a_2 b_0 \\
c_3 &=& a_1 b_2 + a_2 b_1 + a_3 b_0 \\
c_4 &=& a_2 b_2 + a_3 b_1 \\
c_5 &=& a_3 b_2
\end{eqnarray*}
$$

## Part 4: Network Back-Propagation 

![tiny network](https://courses.cs.duke.edu//fall22/compsci371/homework/10/tiny_network.png)

### Problem 4.1 (Exam Style)

### Problem 4.2 (Exam Style)

$$
\begin{eqnarray*}
g_{\hat{y}} &=& \hat{y} - y = 3 - 5 = -2 \\
g_b &=& g_{\hat{y}} \cdot 1 = -2 \\
g_{v_2} &=& g_{\hat{y}}\ z_2 = -2 \cdot 7 = -14\\
g_{v_1} &=&  \\
g_{z_2} &=&  \\
g_{z_1} &=&  \\
g_{a_2} &=&  \\
g_{a_1} &=&  \\
g_{u_{23}} &=&  \\
g_{u_{22}} &=&  \\
g_{u_{21}} &=&  \\
g_{u_{13}} &=&  \\
g_{u_{12}} &=&  \\
g_{u_{11}} &=& 
\end{eqnarray*}
$$

Collecting the relevant entries in reverse order yields

$$
g_{\mathbf{w}} = ()\;.
$$

## Part 5: MNIST Digit Classification

In [13]:
import pickle


file_name = 'mnist.pickle'
retrieve(file_name)
with open(file_name, 'rb') as file:
    mnist = pickle.load(file)

Using previously downloaded file mnist.pickle


In [14]:
def print_result(h, d):
    accuracy = {
        'train': h.score(d['train']['x'], d['train']['y']) * 100,
        'test': h.score(d['test']['x'], d['test']['y']) * 100
    }
    print('training accuracy: {:.2f} percent'.format(accuracy['train']))
    print('test accuracy: {:.2f} percent'.format(accuracy['test']))
    max_points = 20
    p = (accuracy['test'] - 90.) / (96. - 90.) * max_points
    p = min((max_points, max((0, p))))
    p = round(p)
    print('{} out of {} points'.format(p, max_points))

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning
import warnings


def experiment(data, hidden_layer_sizes, max_iter, alpha,
               learning_rate_init, verbose=False):
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        max_iter=max_iter,
        alpha=alpha,
        learning_rate_init=learning_rate_init,
        learning_rate='constant',
        solver='sgd',
        random_state=1,
        verbose=verbose
    )
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore", category=ConvergenceWarning, module="sklearn")
        mlp.fit(data['train']['x'], data['train']['y'])
    print_result(mlp, data)

### Problem 5.1 (Exam Style)

### Problem 5.2

In [16]:
experiment(
    mnist,
    hidden_layer_sizes=(20, 5),
    max_iter=20,
    alpha=1e-8,
    learning_rate_init=0.5,
    verbose=False
)

training accuracy: 10.56 percent
test accuracy: 10.40 percent
0 out of 20 points
